In [2]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:900px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:900px;height:920px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)

###### Constants ######
LEARNING_RATE = 0.00000000003
BATCH_SIZE = 48
DATA_LENGTH = len(housing.data)

###### Implement Data Preprocess here ######
X = tf.placeholder(tf.float32, shape=(BATCH_SIZE, 8), name="X")
Y = tf.placeholder(tf.float32, name="Y")
#w = tf.Variable(np.random.rand(8, 1).astype('f'), name="weights")
w = tf.Variable(np.zeros((8, 1)).astype('f'), name="weights")
b = tf.Variable(0.0, name="bias")

Y_hat = tf.add(tf.matmul(X, w), b)
loss = tf.reduce_sum(tf.square(Y - Y_hat)) / BATCH_SIZE
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

## make mini-batch here
"""
We concat every BATCH_SIZE training sample to form a batch,
to take full GPU parallel processing power, make traing faster.
"""
def make_batch(x, y_hat):
    #initialize one batch
    batch_collection = []
    one_batch = np.zeros((BATCH_SIZE, 8))
    output_collection = []
    one_output = []
    counter = 0
    for d in range(DATA_LENGTH):
        if counter < BATCH_SIZE:
            one_batch[counter] = x[d]
            counter += 1
            one_output.append(y_hat[d])
        else:
            batch_collection.append(one_batch)
            output_collection.append(one_output)
            #initialize one batch again
            one_batch = np.zeros((BATCH_SIZE, 8))
            one_batch[0] = x[d]
            counter = 1
            one_output = [y_hat[d]]
    #push last batch into collection
    batch_collection.append(one_batch)
    output_collection.append(one_output)
    
    return batch_collection, output_collection

## Shuffle Data
"""
Randomly displace every training sample
"""
rnd_order = np.arange(housing.data.shape[0])
np.random.shuffle(rnd_order)
housing.data = housing.data[rnd_order]
housing.target = housing.target[rnd_order]


###### Start TF session ######
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_bat, y_bat = make_batch(housing.data, housing.target)
    
    #record last error rate
    last_err_rate = 99999
    #++ if error rate raising
    raising_count = 0
    best_error_rate = 0
    #record best error rate in which epoch
    best_time = 0
    
    for i in range(1000):
        cost = 0
        for j in range(int(len(x_bat) * 0.9)):
            x = x_bat[j]
            y = y_bat[j]
            sess.run(optimizer, feed_dict={X: x, Y: y}) #actual training
            cost += sess.run(loss, feed_dict={X: x, Y: y}) #calculate cost
        print("========================\nepoch: {0}, \nCost: {1}, \nWeight: {2}, \nBias: {3}".format(i, cost / len(x_bat), sess.run(w).reshape((1, 8)), sess.run(b)))
        
        #test
        err_rate = 0.0
        for k in range(int(len(x_bat) * 0.9), len(x_bat)):
            x = x_bat[k]
            y = y_bat[k]
            y_prid = sess.run(Y_hat, feed_dict={X: x}) #feed testing sample into network
            err_rate += np.sum(np.abs(y - y_prid) / y) / BATCH_SIZE #average error rate
        avg_err_rate = err_rate / len(x_bat) * 0.1 #average overall error rate
        print(avg_err_rate)
        #Early stopping
        """
        if error rate raising consecutively over 10 epoch, 
        we stop training.
        """
        if last_err_rate < avg_err_rate:
            raising_count += 1
            if raising_count >= 10:
                print("Error rate raising, stop.")
                break
        else:
            best_error_rate = avg_err_rate
            best_time = i
            raising_count = 0
        last_err_rate = avg_err_rate
    # report best result
    print("=====Final Result=======")
    print("Best epoch: ", best_time)
    print("Best error rate: ", best_error_rate)
    print("========================")
    show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

Shape of dataset: (20640, 8)
Shape of label: (20640,)
20640
epoch: 0, 
Cost: 149.2855056607446, 
Weight: [[  6.23086180e-06   4.49621984e-05   7.45022635e-06   1.46802176e-06
    9.38390789e-04   3.48745334e-06   4.80389863e-05  -1.61968943e-04]], 
Bias: 1.3519794492822257e-06
0.308269101243
epoch: 1, 
Cost: 131.66114087881044, 
Weight: [[  1.08268268e-05   7.77002424e-05   1.26532677e-05   2.47017783e-06
    9.53459181e-04   5.54648750e-06   8.05192685e-05  -2.71835539e-04]], 
Bias: 2.2685360363539075e-06
0.311897958089
epoch: 2, 
Cost: 130.61626322879349, 
Weight: [[  1.53605670e-05   1.09939931e-04   1.77692455e-05   3.45452418e-06
    9.47658264e-04   7.55715109e-06   1.12417540e-04  -3.79754434e-04]], 
Bias: 3.1688209674030077e-06
0.311077700875
epoch: 3, 
Cost: 129.56686752341514, 
Weight: [[  1.98678754e-05   1.41948447e-04   2.28473455e-05   4.43126282e-06
    9.41428530e-04   9.55083215e-06   1.44074656e-04  -4.86867037e-04]], 
Bias: 4.062375410285313e-06
0.310194017989
epoch:

epoch: 32, 
Cost: 104.88171080434046, 
Weight: [[  1.40234319e-04   9.79268923e-04   1.55262009e-04   2.97772694e-05
    7.78795511e-04   6.07930670e-05   9.67871747e-04  -3.27794277e-03]], 
Bias: 2.7344223781256005e-05
0.300818608087
epoch: 33, 
Cost: 104.20176979552868, 
Weight: [[  1.44053774e-04   1.00523187e-03   1.59352887e-04   3.05559406e-05
    7.73774227e-04   6.23496308e-05   9.93262860e-04  -3.36410059e-03]], 
Bias: 2.8062850105925463e-05
0.300965541437
epoch: 34, 
Cost: 103.53159832510838, 
Weight: [[  1.47852814e-04   1.03101507e-03   1.63414399e-04   3.13287564e-05
    7.68789439e-04   6.38932470e-05   1.01846666e-03  -3.44963721e-03]], 
Bias: 2.8776266844943166e-05
0.301135856223
epoch: 35, 
Cost: 102.87107063005136, 
Weight: [[  1.51631757e-04   1.05662108e-03   1.67446764e-04   3.20957042e-05
    7.63840799e-04   6.54239484e-05   1.04348618e-03  -3.53455241e-03]], 
Bias: 2.948454493889585e-05
0.301330221565
epoch: 36, 
Cost: 102.22005009540291, 
Weight: [[  1.55390589

In [8]:
sess